In [1]:
import pandas as pd
from database.market import Market
from datetime import datetime,timedelta
from parameters.parameters import Parameters
from backtester.backtester_lite import BacktesterLite
from processor.processor import Processor as p
from tqdm import tqdm
import matplotlib.pyplot as plt
import json

In [2]:
market = Market()

In [3]:
market.connect()
sp100 = market.retrieve("sp100")
sp100 = market.retrieve("sp100")
market.disconnect()

In [4]:
start_date = datetime.now() - timedelta(days=7*365.25)
end_date = datetime.now()
lookbacks = [5]
holding_periods = [1]
floors = [-10,0]
ceilings = [1,10]
local_min = [False]
volatilities = [0.1,0.5]
positions = 11
tickers = sp100["ticker"]
asset = "stocks"
positions = len(list(sp100["GICS Sector"].unique()))

In [5]:
parameters = Parameters.parameters_lite(lookbacks,holding_periods,ceilings,floors,volatilities,local_min)

In [6]:
simulation = []
market.connect()
for ticker in tqdm(tickers):
    try:
        ticker_data = market.retrieve_ticker_prices(asset,ticker)
        ticker_data = p.column_date_processing(ticker_data)
        ticker_data.sort_values("date",inplace=True)
        ticker_data["week"] = [x.week for x in ticker_data["date"]]
        ticker_data["day"] = [x.weekday() for x in ticker_data["date"]]
        ticker_data["prev_close"] = ticker_data["adjclose"].shift(1)
        for lookback in lookbacks:
            ticker_data[f"window_{lookback}"] = ticker_data["prev_close"].shift(lookback)
            ticker_data[f"rolling_{lookback}"] = ticker_data["prev_close"].rolling(lookback).mean()
            ticker_data["d1"] = ticker_data[f"adjclose"].pct_change()
            ticker_data[f"rolling_stdev_{lookback}"] = ticker_data["prev_close"].rolling(lookback).std()
            ticker_data[f"rolling_pct_stdev_{lookback}"] = ticker_data[f"rolling_stdev_{lookback}"] / ticker_data[f"rolling_{lookback}"]
        for holding_period in holding_periods:
            ticker_data[f"return_{holding_period}"] = (ticker_data["adjclose"].shift(-holding_period) - ticker_data["adjclose"]) / ticker_data["adjclose"]
        simulation.append(ticker_data.dropna())
    except Exception as e:
        print(ticker,str(e))
market.disconnect()
final = pd.concat(simulation)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 503/503 [00:11<00:00, 44.56it/s]


In [7]:
final = pd.concat(simulation).merge(sp100[["ticker","GICS Sector"]],how="left")

In [8]:
all_trades = []
for iteration in tqdm(range(len(parameters))):
    try:
        parameter = parameters[iteration]
        trades = BacktesterLite.backtest(sp100,final.copy(),iteration,parameter,False)
        all_trades.append(trades)
    except Exception as e:
        print(str(e))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:24<00:00,  1.32it/s]


In [9]:
analysis = pd.concat(all_trades)
analysis["return"] = analysis["return"] + 1

In [10]:
report = []
for iteration in tqdm(range(len(parameters))):
    try:
        parameter = parameters[iteration]
        iteration_trades = analysis[analysis["iteration"]==iteration]
        iteration_trades = iteration_trades[(iteration_trades["date"]>=start_date) & (iteration_trades["date"]<=end_date)]
        cumulative = iteration_trades.pivot_table(values="return",index="date",columns="position").reset_index()
        for i in range(positions):
            if float(i) not in cumulative.columns:
                cumulative[float(i)] = 1
        cumulative["pv"] = [sum([row[1][float(x)] * float(1/positions) for x in range(positions)]) for row in cumulative.iterrows()]
        iter_report = cumulative.cumprod().iloc[-1]
        for key in parameter.keys():
            iter_report[key] = parameter[key]
        iter_report["iteration"] = iteration
        report.append(iter_report[["pv","iteration"] + list(parameter.keys())])
    except Exception as e:
        print(str(e))
        continue

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:03<00:00, 10.01it/s]


In [11]:
iteration_trades

,date,ticker,signal,return,position,iteration,strategy,value,lookback,holding_period,floor,ceiling,volatility,local_min,industry_weighted,weekend,constituent
0,2016-08-30 04:00:00,HST,0.019989,0.985075,0,31,window,True,5,1,0,10,0.5,False,True,True,500
1,2016-08-30 04:00:00,PYPL,0.021990,1.004054,1,31,window,True,5,1,0,10,0.5,False,True,True,500
2,2016-08-30 04:00:00,CTRA,0.023052,0.991546,2,31,window,True,5,1,0,10,0.5,False,True,True,500
3,2016-08-30 04:00:00,MTCH,0.026958,1.021451,3,31,window,True,5,1,0,10,0.5,False,True,True,500
4,2016-08-30 04:00:00,ENPH,0.032609,0.989130,4,31,window,True,5,1,0,10,0.5,False,True,True,500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15008,2023-08-17 04:00:00,PODD,0.090646,0.984709,4,31,window,True,5,1,0,10,0.5,False,True,True,500
15009,2023-08-17 04:00:00,SEE,0.122374,0.974272,5,31,window,True,5,1,0,10,0.5,False,True,True,500
15010,2023-08-17 04:00:00,SEDG,0.128204,0.997358,6,31,window,True,5,1,0,10,0.5,False,True,True,500
15011,2023-08-17 04:00:00,DFS,0.156215,1.010392,7,31,window,True,5,1,0,10,0.5,False,True,True,500


In [12]:
performance = pd.DataFrame(report)

In [13]:
performance.sort_values("pv",ascending=False).head(5)

position,pv,iteration,strategy,value,lookback,holding_period,floor,ceiling,volatility,local_min,industry_weighted,weekend,constituent
1400,12.076881,26,window,True,5,1,0,1,0.1,False,True,True,500
1400,11.907017,30,window,True,5,1,0,10,0.1,False,True,True,500
1400,11.026506,27,window,True,5,1,0,1,0.5,False,True,True,500
1400,9.966882,31,window,True,5,1,0,10,0.5,False,True,True,500
1400,9.357163,24,window,True,5,1,-10,1,0.1,False,True,True,500


In [14]:
iteration = performance.sort_values("pv",ascending=False).iloc[0]["iteration"]
parameter = parameters[iteration]
iteration_trades = analysis[analysis["iteration"]==iteration]
iteration_trades = iteration_trades[(iteration_trades["date"]>=start_date) & (iteration_trades["date"]<=end_date)]
cumulative = iteration_trades.pivot_table(values="return",index="date",columns="position").reset_index()
cumulative["pv"] = [sum([row[1][float(x)] * float(1/positions) for x in range(positions)]) for row in cumulative.iterrows()]
cumulative["pv"] = cumulative["pv"].cumprod()

In [15]:
# cumulative

In [16]:
# plt.plot(cumulative["date"],cumulative["pv"])

In [17]:
pd.DataFrame([parameter]).to_csv("parameter.csv")